<a href="https://colab.research.google.com/github/danielsteman/textifai/blob/main/PDF_to_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdf2image layoutparser pymupdf torchvision && pip install "git+https://github.com/facebookresearch/detectron2.git@v0.5#egg=detectron2"	
!apt-get install poppler-utils 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 878.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 1.3 MB/s eta 0:00:00
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31531 sha256=aa254cfa6bbb7993c2959b820edc6ff48666d905a0cb437a149b2e1980d879ec
  Stored in directory: /root/.cache/pip/wheels/9a/a3/b6/ac0fcd1b4ed5cfeb3db92e6a0e476cfd48ed0df92b91080c1d
Successfully built iopath
  Attempting u

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 24 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 1s (117 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 122518 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.86.1-0ubuntu1.1_amd64.deb ...
Unpacking poppler-utils (0.86.1-0ubuntu1.1) ...
Setting up poppler-utils (0.86.1-0ubuntu1.1) ...
Processing triggers for man-db (2.9.1-1) ...


In [ ]:
import pdf2image
import numpy as np
import layoutparser as lp
import fitz
import time
import sys
import re
import os
import collections
import multiprocessing

In [ ]:
pdf_file = 'TextRank Bringing Order into Texts.pdf'

In [ ]:
doc = fitz.open(pdf_file)

In [ ]:
pages = doc.page_count

In [ ]:
model = lp.Detectron2LayoutModel('lp://PubLayNet/mask_rcnn_X_101_32x8d_FPN_3x/config',
                                 extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                                )

config.yaml?dl=1: 8.19kB [00:02, 2.89kB/s]
model_final.pth?dl=1: 856MB [00:57, 15.0MB/s]                           
  proposal_generator.anchor_generator.cell_anchors.{0, 1, 2, 3, 4}


In [ ]:
model.label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"}

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

%pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.6 MB/s eta 0:00:00


In [ ]:
import re
import spacy
from PyPDF2 import PdfReader

# Load the small English model in spaCy
# You need to install spaCy and download the model: pip install spacy && python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

def is_front_page(text):
    # Patterns to search for in the text
    title_pattern = re.compile(r'\btitle\b', re.IGNORECASE)
    author_pattern = re.compile(r'\bauthor(s)?\b', re.IGNORECASE)
    abstract_pattern = re.compile(r'\babstract\b', re.IGNORECASE)

    # Check if the patterns are found in the text
    has_title = bool(title_pattern.search(text))
    has_author = bool(author_pattern.search(text))
    has_abstract = bool(abstract_pattern.search(text))

    # Analyze text with spaCy
    doc = nlp(text)

    # Check for common entities found on front pages
    has_person = any([ent.label_ == "PERSON" for ent in doc.ents])
    has_org = any([ent.label_ == "ORG" for ent in doc.ents])

    # Classify as front page if at least two of the patterns are found, or if it has person and organization entities
    return ((has_title and has_author) or (has_title and has_abstract) or (has_author and has_abstract)) or (has_person and has_org)

# Example 1 usage
pdf_file1 = 'Textrank Bringing Order into Texts.pdf'
with open(pdf_file1, 'rb') as f:
    # Extract text from the first page of the PDF
    reader = PdfReader(f)
    first_page = reader.pages[0]
    first_page_text = first_page.extract_text()

    # Check if it's a front page
    is_front = is_front_page(first_page_text)
    print('Front page:', is_front)

# Example 2 usage
pdf_file2 = 'MASTERING DIGITAL.pdf'
with open(pdf_file2, 'rb') as f:
    # Extract text from the first page of the PDF
    reader = PdfReader(f)
    first_page = reader.pages[0]
    first_page_text = first_page.extract_text()

    # Check if it's a front page
    is_front = is_front_page(first_page_text)
    print('Front page:', is_front)

Front page: True
Front page: True


In [ ]:
with open(pdf_file, 'rb') as f:
    # Extract text from the first page of the PDF
    # You need to install PyPDF2 for this: pip install PyPDF2
    import PyPDF2

    reader = PyPDF2.PdfReader(f)
    first_page = reader.pages[0]
    first_page_text = first_page.extract_text()

    # Check if it's a front page
    is_front = is_front_page(first_page_text)
    print('Front page:', is_front)

Front page: False


In [ ]:
def analyze_pdf_columns(file_path):
    """
    Analyzes the layout of a PDF file to determine the number of columns on the first page.
    
    Args:
        file_path (str): The path to the PDF file.
    
    Returns:
        int: The number of columns detected on the first page of the PDF.
    """
    
    doc = fitz.open(file_path)
    page = doc.load_page(1)  # analyze the first page
    blocks = page.get_text("blocks")
    
    # Group text blocks by their vertical positions
    y_positions = collections.defaultdict(list)
    for block in blocks:
        x0, y0, x1, y1, *_ = block
        y_positions[int(y0)].append(block)
    
    # Count the number of columns based on the number of text blocks in each horizontal line
    columns_count = max([len(y_positions[key]) for key in y_positions])

    return columns_count

In [ ]:
def convert_dpi_to_coordinates(dpi, target_size):
    
    """
    Converts DPI values to corresponding scale factors and margins for resizing coordinates.
    
    Args:
        dpi (tuple): A tuple containing the DPI values for the x and y axes (width, height).
        target_size (tuple): A tuple containing the target size of the output image (width, height).
    
    Returns:
        tuple: A tuple containing the x_margin, y_margin, x_scale, and y_scale values.
    """

    x_margin = target_size[0]/100*1
    y_margin = target_size[0]/100*1

    x_scale = target_size[0] / dpi[0]
    y_scale = target_size[1] / dpi[1]
    
    return x_margin, y_margin, x_scale, y_scale

In [ ]:
def convert_and_scale_coordinates(dpi, target_size, text_blocks):
    """
    Converts DPI values to corresponding scale factors and margins for resizing coordinates and scales the coordinates.
    
    Args:
        dpi (tuple): A tuple containing the DPI values for the x and y axes (width, height).
        target_size (tuple): A tuple containing the target size of the output image (width, height).
        text_blocks (list): A list of text blocks with coordinates.
    
    Returns:
        list: A list of tuples containing the scaled coordinates.
    """
    x_margin = target_size[0]/100*1
    y_margin = target_size[0]/100*1

    x_scale = target_size[0] / dpi[0]
    y_scale = target_size[1] / dpi[1]
    
    # Resize coordinates and margins
    print("===== Convert coordinates ====")
    x_margin, y_margin, x_scale, y_scale = convert_dpi_to_coordinates(dpi, target_size)

    scaled_coordinates = []

    for block in text_blocks:
        c = block.block.coordinates
        scaled_coordinates.append((c[0]*x_scale-x_margin, c[1]*y_scale-y_margin, c[2]*x_scale+x_margin, c[3]*y_scale+y_margin))
    
    return scaled_coordinates

In [ ]:
def process_text_fields(layout_result, img, img_width):
    """
    Process text fields from a layout result and sort them based on their position on the page.

    Args:
        layout_result (lp.Layout): A layout result object containing the layout data.
        img_width (int): The width of the image.

    Returns:
        lp.Layout: A layout object containing the processed and sorted text blocks.
    """
    
    # Extract and process Text fields
    text_blocks = lp.Layout([b for b in layout_result if b.type == 'Text'])

    left_interval = lp.Interval(0, img_width / 2 * 1.05, axis='x').put_on_canvas(img)

    left_blocks = text_blocks.filter_by(left_interval, center=True)
    left_blocks.sort(key=lambda b: b.coordinates[1], inplace=True)

    right_blocks = lp.Layout([b for b in text_blocks if b not in left_blocks])
    right_blocks.sort(key=lambda b: b.coordinates[1], inplace=True)

    # Combine the two lists and add the index
    text_blocks = lp.Layout([b.set(id=idx) for idx, b in enumerate(left_blocks + right_blocks)])

    return text_blocks

In [ ]:
def get_text_from_pdf(page, model=model, pdf=pdf_file):  
    
    """
    Extracts text from a specified PDF page using a given layout detection model.
    
    Args:
        page (int): The page number to extract text from.
        model: The layout detection model.
        pdf_path (str): The path to the PDF file.
        dpi (tuple): The DPI of the input image (default: (1700, 2200)).
        target_size (tuple): The target size of the PDF page (default: (612, 792)).
    
    Returns:
        list: A list of extracted text blocks.
    """
    
    # Load specified page
    p = doc.load_page(page) 
    # PDF Size
    pdf_h, pdf_w = p.rect.width, p.rect.height
    
    # Convert PDF to IMG
    print("===== Convert page to img ====")
    img = np.asarray(pdf2image.convert_from_path(pdf_path=pdf)[page])
    # DPI Dimensions 
    img_h, img_w = img.shape[:2]
    
    # Document Layout Analysis to extract Text fields only
    print("===== Run LayoutDetection Model ====")
    layout_result = model.detect(img)
    print("===== Finished LayoutDetection Model ====")

    # Extract text blocks
    text_blocks = process_text_fields(layout_result=layout_result, img=img, img_width=img_w)

    # Resize coordinates and margins
    print("===== Convert coordinates ====")

    scaled_coordinates = convert_and_scale_coordinates(dpi=(1700, 2200), target_size=(612, 792), text_blocks=text_blocks)

    # Extract Text blocks
    print("===== Extract text ====")

    text = []

    for sc in scaled_coordinates:
        rct = fitz.Rect(sc)
        txt = p.get_textbox(rct)
        text.append(txt)  
    
    print("===== Done ====")
    
    return text

In [ ]:
def get_pdf_dimensions(pdf_file_path):
    """
    Get the height and width of a PDF file using the PyMuPDF library.

    Args:
        pdf_file_path (str): The path to the PDF file.

    Returns:
        tuple: A tuple containing the width and height of the PDF file.
    """
    doc = fitz.open(pdf_file_path)
    page = doc.load_page(0)  # Load the first page of the document
    width, height = page.rect.width, page.rect.height
    doc.close()

    return width, height

In [ ]:
def parallel_extract_text(num_processes=None):
    """
    Extract text from each page of a PDF file in parallel using multiprocessing.

    Args:
        pdf_path (str): Path to the PDF file to be processed.
        num_processes (int, optional): Number of processes to use. Defaults to the number of available CPU cores.

    Returns:
        list: Extracted text from each page of the PDF in the order they appear.
    """
    global pdf_file

    with fitz.open(pdf_file) as doc:
        num_pages = doc.page_count

    if num_processes is None:
        num_processes = multiprocessing.cpu_count()

    # Create a temporary file to avoid opening the PDF file multiple times
    with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
        tmp_file.write(open(pdf_file, 'rb').read())
        tmp_pdf_path = tmp_file.name

    try:
        with multiprocessing.Pool(processes=num_processes) as pool:
            # Process each page in parallel
            results = pool.map(get_text_from_pdf, [(tmp_pdf_path, i, model) for i in range(num_pages)])
    finally:
        os.unlink(tmp_pdf_path)  # Remove the temporary file

    return results

In [ ]:
parallel_extract_text(num_processes=8)

ValueError: ignored

In [ ]:
s = time.time()
text = []

for page in range(pages):
  text.append(get_text_from_pdf(page))

e = time.time()

print(f"Operation took {e-s} seconds")

===== Convert page to img ====
===== Run LayoutDetection Model ====


/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


===== Finished LayoutDetection Model ====
===== Convert coordinates ====
===== Convert coordinates ====
===== Extract text ====
===== Done ====
===== Convert page to img ====
===== Run LayoutDetection Model ====
===== Finished LayoutDetection Model ====
===== Convert coordinates ====
===== Convert coordinates ====
===== Extract text ====
===== Done ====
===== Convert page to img ====
===== Run LayoutDetection Model ====
===== Finished LayoutDetection Model ====
===== Convert coordinates ====
===== Convert coordinates ====
===== Extract text ====
===== Done ====
===== Convert page to img ====
===== Run LayoutDetection Model ====
===== Finished LayoutDetection Model ====
===== Convert coordinates ====
===== Convert coordinates ====
===== Extract text ====
===== Done ====
===== Convert page to img ====
===== Run LayoutDetection Model ====
===== Finished LayoutDetection Model ====
===== Convert coordinates ====
===== Convert coordinates ====
===== Extract text ====
===== Done ====
===== Co

In [ ]:
text

[['Rada Mihalcea and Paul Tarau\nDepartment of Computer Science\nUniversity of North Texas\n� rada,tarau\n� @cs.unt.edu',
  'Abstract\nIn this paper, we introduce TextRank – a graph-based\nranking model for text processing, and show how this\nmodel can be successfully used in natural language\napplications. In particular, we propose two innova-\ntive unsupervised methods for keyword and sentence\nextraction, and show that the results obtained com-\npare favorably with previously published results on\nestablished benchmarks.',
  '1\nIntroduction\nGraph-based ranking algorithms like Kleinberg’s\nHITS algorithm (Kleinberg,\n1999) or Google’s\nPageRank (Brin and Page, 1998) have been success-\nfully used in citation analysis, social networks, and\nthe analysis of the link-structure of the World Wide\nWeb. Arguably, these algorithms can be singled out\nas key elements of the paradigm-shift triggered in\nthe field of Web search technology, by providing a\nWeb page ranking mechanism that reli

In [ ]:
import re

def clean_text(text):
    # Remove non-ASCII characters
    ascii_text = re.sub(r'[^\x00-\x7F]+', ' ', text)

    # Replace newline characters and multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', ascii_text).strip()

    return cleaned_text

# Sample extracted text
text = '''of two sentences with the length of each sentence.\nFormally, given two sentences\n�\n� and\n�\nsentence being represented by the set of� , with a\n�\n� words\nthat appear in the sentence:\n�\n�\n�\n�\n�\n�\n���\n�\n�\n����\n���\n��\n$ ,\nthe similarity of\n�\n� and\n�� is defined as:',
'������������������K���\n�\n���\n�\n�2�\n4\n!#"�$�%\n�\n�\n$\n�\n&�\'\n!#"�$�%\n�\n�\n4\n�\n&\nOther sentence similarity measures, such as string\nkernels, cosine similarity, longest common subse-\nquence, etc. are also possible, and we are currently\nevaluating their impact on the summarization perfor-\nmance.\nThe resulting graph is highly connected'''

cleaned_text = clean_text(text)
print(cleaned_text)


of two sentences with the length of each sentence. Formally, given two sentences and sentence being represented by the set of , with a words that appear in the sentence: $ , the similarity of and is defined as:', ' K 2 4 !#" $ % $ & ' !#" $ % 4 & Other sentence similarity measures, such as string kernels, cosine similarity, longest common subse- quence, etc. are also possible, and we are currently evaluating their impact on the summarization perfor- mance. The resulting graph is highly connected
